# Load the dataset

In [ ]:
from dataportal import DataportalClient

token = ''

In [ ]:
help(DataportalClient)

In [ ]:
client = DataportalClient(token)
files = client.fromDataset('5Gdata').listFiles()

In [ ]:
df = client.getData(list(files)[0]['FileID'])

In [ ]:
df.to_csv("CellMeas.csv")

# Let's operate on the dataset here below!

In [ ]:
!{sys.executable} -m pip -q install xgboost sklearn_evaluation

In [ ]:
# python modules
import io
import os
import datetime

# external modules
#import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.colors import ListedColormap
from sklearn_evaluation import plot

In [ ]:
def load_dataset():
    # see if filtered file exists:
    try:
        df = pd.read_csv("filtered.csv", low_memory=False)
        return df
    except Exception:
        print("Need to load full file")

    # fix dataset
    splits = {}
    with open("CellMeas.csv") as f:
        splitNo = 0
        currentSplit = None
        lines = f.readlines()
        # print(len(lines))
        k = 0
        for line in lines:
            k += 1
            # print(l)
            if line.find("name") >= 0:
                # print("new file")
                splitNo += 1
                name = f"split{splitNo}"

                currentSplit = io.StringIO()
                splits[name] = currentSplit

            if currentSplit is not None:
                currentSplit.write(line)

    dsplits = {}
    for name, f in splits.items():
        f.seek(0)
        data = pd.read_csv(f)
        f.close()
        dsplits[name] = data

    df = None
    for name, d in dsplits.items():
        if df is None:
            df = d
        else:
            df = pd.concat([df, d])

    try:
        df.to_csv("filtered.csv")
    except Exception:
        print("Failed to write filtered dataframe")

    return df

In [ ]:
def prune_dataset(data):
    data = data.dropna(axis=1)
    # Remove static device info
    data = data.loc[:,~data.columns.str.startswith('DEVICE_INFO')]

    # drop the "topic"
    # data = data.drop(columns=['topic'])
    data = data.drop(columns=['DATA_FORMAT_VERSION'])
    #print(data.head())
    # df = drop_strange_values(df, [2147483647])

    return data

In [ ]:
def add_features(data):
    # time in this data is in nanosecond
    # timestamp_nanoseconds = 1644412586263150848.00000
    # timestamp_seconds = timestamp_nanoseconds / 1000000000.0
    # Replace the timestamp_seconds with your Unix timestamp in seconds
    #data['time_second'] = (data['time']/1000000000).round()
    data['time_second'] = (data['mTimeStamp']/1000000000).round()
    data = data.drop(['mTimeStamp', 'time'], axis=1)
    data['date'] = pd.to_datetime(data['time_second'], unit='s')
    #print(data['date'])
    #quit()
    #print('data shape:', data.shape)
    #data_downsampled = data.drop_duplicates(subset = 'time_second') # keep only one fo the samples for the same time in second
    #data_downsampled = data_downsampled.set_index('date')
    #data_downsampled = data_downsampled.sort_index()
    #print('data downsampled shape:', data_downsampled.shape)
    #plt.plot(data_downsampled['mPingLoss'])
    #plt.ylim(0, 250)
    #plt.xlim(0, 1000)
    #plt.savefig("wasp_pingloss_ds.pdf")
    #plt.savefig("wasp_pingloss_ds.png")
    #plt.show()

    data['second'] = data.date.dt.second
    data['minute'] = data.date.dt.minute
    data['hour'] = data.date.dt.hour
    data['month'] = data.date.dt.month
    data['day_of_month'] = data.date.dt.day
    data['day_of_year'] = data.date.dt.dayofyear
    # data['week_of_year'] = data.date.dt.weekofyear
    data['day_of_week'] = data.date.dt.dayofweek
    # data['year'] = data.date.dt.year
    data["is_wknd"] = data.date.dt.weekday // 4
    data['is_month_start'] = data.date.dt.is_month_start.astype(int)
    data['is_month_end'] = data.date.dt.is_month_end.astype(int)
    data = data.drop(['date'], axis=1)

    # Add a new column 'ChangeIndicator' with 1 when CellID changes and 0 otherwise
    data['ChangeIndicator_CellID'] = (data['mCellID'] != data['mCellID'].shift()).astype(int)
    # Add new column 'Loss' with 1 when the Ping loss is  not zero otherwise when the ping loss is zero it mean there is no loss
    data['Loss'] = (data['mPingLoss'] != 0).astype(int)
    data = data.drop(['mPingLoss'], axis=1)

    return data

In [ ]:
def one_hot_encode(data):
    df_categorical_features = data.select_dtypes(include='object')
    cat_columns = list(df_categorical_features.columns)

    # print("Categorical data: {}".format(cat_columns))

    # df = df.select_dtypes(exclude='object')
    df_categorical_features = data.select_dtypes(include='object')
    cat_columns = list(df_categorical_features.columns)
    # print("Categorical data: {}".format(cat_columns))

    df_encoded = pd.get_dummies(data, columns=cat_columns, dtype=int)

    return df_encoded

In [ ]:
def correlation_plot(data, feature):
    # Calculating the correlation matrix
    correlation_matrix = data.corr()

    # Finding highly correlated features with target
    highly_correlated_features = correlation_matrix[feature][(correlation_matrix[feature] > 0.4) & (correlation_matrix[feature] < 1.0)]
    # Adjust the threshold values (0.4 and 1.0) according to your desired level of correlation

    print(f'Highly correlated features with "{feature}":')
    print(highly_correlated_features)

    # Filter the highly correlated features
    highly_correlated_data = data[highly_correlated_features.index]

    # Plotting the correlation matrix of highly correlated features
    plt.figure(figsize=(8, 6))
    sns.heatmap(highly_correlated_data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(f'Correlation Matrix of Highly Correlated Features with "{feature}"')

    # Save the figure as an image file (e.g., PNG, JPG, PDF, etc.)
    plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')  # Change file format as needed
    plt.show()

In [ ]:
def drop_strange_values(data, values):
    # bad_cols = df.columns[data.eq(2147483647).any()]
    bad_cols = data.columns[data.isin(values).any()]
    for bc in bad_cols:
        for v in values:
            data = data[data[bc] != v]

    return data

In [ ]:
def train_model(data, feature, analysis):
    """
    analysis = "pca" | "regions"
    """
    y = data[feature]
    X = data.drop([feature], axis=1)

    print('y shape {}'.format(str(y.shape)))
    print('X shape {}'.format(str(X.shape)))

    # ...the class column has to start from 0 (as required since version 1.3.2). An easy way to solve that is using LabelEncoder from sklearn.preprocssing library.
    le = LabelEncoder()
    y = le.fit_transform(y)

    # Assuming you have features (X) and labels (y)
    # Splitting for each target variable individually
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f'Machine learning model on feature "{feature}"')
    # Parameters:
    # n_estimators (Optional[int]) – Number of boosting rounds.
    # max_depth (Optional[int]) – Maximum tree depth for base learners.
    # max_leaves – Maximum number of leaves; 0 indicates no limit.
    # max_bin – If using histogram-based algorithm, maximum number of bins per feature
    # grow_policy – Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow depth-wise. 1: favor splitting at nodes with highest loss change.
    # learning_rate (Optional[float]) – Boosting learning rate (xgb’s “eta”)
    # ......
    model = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
    model.fit(X_train, y_train)
    print('Accuracy of XGB loss classifier on training set: {:.4f}'.format(model.score(X_train,y_train)))
    print('Definitions:  Precision: (true positives) / (true positives + false positives)')
    print('              Recall   : (true positives) / (true positives + false negatives)')

    predictions = model.predict(X_test)
    classification = classification_report(y_test, predictions, zero_division=np.nan)
    print(classification)


    # Plots
    figname = ""
    title = ""

    if analysis == "pca":
        # Classic PCA analysis

        orig = MinMaxScaler().fit_transform(X_train)
        # 99% of the variance of the original features has been retained
        # whiten=True transforms the values of each principal component so that they have 0 mean and 1 variance
        pca_data = PCA(n_components=0.99, whiten=True)
        pca_fitness = pca_data.fit_transform(orig)
        #sns.scatterplot(pca_fitness[:,0], pca_fitness[:,1])
        print("shape of pca_fitness: {}".format(str(pca_fitness.shape)))
        plt.scatter(pca_fitness[:,0], pca_fitness[:,1])

        title = f'PCA analysis on "{feature}" model'
        figname = f"wasp_pca_model_{feature}.png"

    if analysis == "regions":
        title = f'Decisions regions on "{feature}" model'
        figname = f"wasp_regions_model_{feature}.png"
        plot_decision_regions(X_train, y_train)

    if figname != "":
        plt.tight_layout()
        plt.title(title)
        plt.savefig(f"wasp_shap_model_{feature}.png", bbox_inches='tight')
        #plt.show()

    y_score = model.predict_proba(X_test)

    return y_test, predictions, y_score

In [ ]:
def plot_decision_regions(X, y, test_idx=None, resolution=0.02):
    # make reduction of space
    orig = MinMaxScaler().fit_transform(X)
    # 99% of the variance of the original features has been retained
    # whiten=True transforms the values of each principal component so that they have 0 mean and 1 variance
    pca_data = PCA(n_components=2, whiten=True)
    reducedX = pca_data.fit_transform(orig)

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')

    #color = cm.nipy_spectral(float(i) / n_clusters).reshape(1,-1)
    cmap = ListedColormap(colors[:len(np.unique(y))])
    print(cmap)
    # plot the decision surface
    x1_min, x1_max = reducedX[:, 0].min() - 1, reducedX[:, 0].max() + 1
    x2_min, x2_max = reducedX[:, 1].min() - 1, reducedX[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))

    # make a model with the reduced space
    X_train, X_test, y_train, y_test = train_test_split(reducedX, y, test_size=0.3)
    xgb_clf = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
    xgb_clf = xgb_clf.fit(X_train, y_train)

    Z = xgb_clf.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)

    plt.contourf(xx1, xx2, Z, alpha=0.4)  #, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=reducedX[y == cl, 0], y=reducedX[y == cl, 1],
                    alpha=0.8,
                    #c=cmap(idx),
                    marker=markers[idx],
                    label=cl)

    # highlight test samples
    if test_idx is not None:
        # plot all samples

        #X_test, y_test = reducedX[test_idx, :], y[test_idx]

        plt.scatter(reducedX[:, 0],
                    reducedX[:, 1],
                    c='',
                    alpha=1.0,
                    linewidths=1,
                    marker='o',
                    s=55, label='test set')

    #plt.show()

# Prepare the dataset
The dataset will be loaded and cleaned according to some heuristics. Some categorical data (mainly static) is removed and columns containing ```NaN``` are removed. The remaining categorical data is one-hot encoded. Features related to time is added as well as making some columns binary (reducing all positive numbers to ```1```).

In [ ]:
# This file contains several CSV files. For some reason. Clean it first!
df = load_dataset()

# Let's save some time and memory, just keep parts of it?
df = df.head(400000)

print("Shape of data prior to pruning: {}".format(str(df.shape)))
df = prune_dataset(df)

print("Shape of data prior to adding features: {}".format(str(df.shape)))
df = add_features(df)

print("Shape of data prior to one-hot-encoding: {}".format(str(df.shape)))
df = one_hot_encode(df)

# print("Currently available columns after hot-encoding: {}".format(list(df.columns)))

In [ ]:
df.head()

# Produce Correlation Plots

Compute pairwise correlation of columns in the dataset. Find the features that correlate the most to the selected feature and inspect how the features in this set are correlated.

In [ ]:
# correlation
# 'Unnamed: 0', 'time', 'DATA_FORMAT_VERSION', 'mAsuLevel', 'mBW', 'mCellID', 'mCellInfoTS', 'mCellPci', 'mCellSig', 'mCellTac', 'mCqi', 'mCsiSinr', 'mEarcfn', 'mLevel', 'mLocAcc', 'mLocAlt', 'mLocLat', 'mLocLon', 'mNumPings', 'mPingAge', 'mPingAvg', 'mPingLoss', 'mPingMax', 'mPingMin', 'mRegOnCell', 'mRsrp', 'mRsrq', 'mRssi', 'mRssnr', 'mSpeed', 'mSsRsrp', 'mSsRsrq', 'mSsSinr', 'mTimeStamp', 'mTimingAdvance', 'name_CellMeas', 'mCellConn_NONE', 'mCellConn_REQUIRES SDK 28', 'mCellType_GSM', 'mCellType_LTE', 'mCellType_NR', 'mCellType_WCDMA', 'mEventType_CellInfo', 'mEventType_SignalStrength'
# correlation_plot(df, 'Loss')
# correlation_plot(df, 'mPingAge')
# correlation_plot(df, 'mPingAvg')
correlation_plot(df, 'mCqi')

# Apply ML models to perform inference on features
Here, for example, ```Loss``` refers to the existence of ping loss and ```ChangeIndicator_CellID``` refers to handover between mobile cells.

Parameter ```analyis``` can be either ```"pca"``` or ```"regions"```, where

* Principal Component Analysis (PCA)
    * A method that helps to interprete large datasets containing a high number of dimensions/features per observation by reducing the domanisonality while preserving the maximum amount of information.
* regions
    * Plots the deciions regions for the model.

In [ ]:
y_test, y_pred, y_score = train_model(df, 'Loss', analysis='pca')

# A confusion matrix represents the prediction summary in matrix form

In [ ]:
# plot confusion matrix
forest_cm = plot.ConfusionMatrix.from_raw_data(y_test, y_pred)

# Plot Precision/Recall

## Precision

Precision (also called positive predictive value) is the fraction of relevant instances among the retrieved instances. Written as

$$ \text{Precision} = {\text{Relevant retrieved instances} \over \text{All retrieved instances}} = { \text{True positive} \over \text{True positive + False positive} } $$



## Recall
Recall (also known as sensitivity) is the fraction of relevant instances that were retrieved. Written as

$$ \text{Recall} = { \text{Relevant retrieved instances} \over \text{All relevant instances} } = { \text{True positive} \over \text{True positive + False negative} } $$

In [ ]:
# plot Precision/Recall
pr = plot.PrecisionRecall.from_raw_data(y_test, y_score)

# Plot Receiver Operating Characteristic (ROC) curve
The ROC curve is the plot of the true positive rate (TPR) against the false positive rate (FPR) at each threshold setting. Strive for being  on the "upper left"!

In [ ]:
# plot the roc curve
roc = plot.ROC.from_raw_data(y_test, y_score)